<a href="https://colab.research.google.com/github/ericbonelli/Cientista-de-Dados_EBAC/blob/main/GBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Gradient Boosting Machine (GBM)

---

## ✅ 1. Cinco diferenças entre AdaBoost e GBM

| Característica             | AdaBoost                                             | GBM (Gradient Boosting Machine)                          |
|----------------------------|------------------------------------------------------|-----------------------------------------------------------|
| Atualização dos pesos     | Ajusta pesos dos exemplos com base nos erros        | Minimiza função de perda diretamente                      |
| Tipo de função de perda   | Usa erro classificatório                            | Pode usar MSE, MAE, LogLoss, etc. (mais flexível)         |
| Algoritmo de aprendizado  | Sequencial adaptativo                               | Otimização via gradiente da função de perda               |
| Sensibilidade a outliers  | Alta (por causa dos pesos)                          | Pode ser ajustada com `loss` e `learning_rate`            |
| Nome alternativo          | Adaptive Boosting                                   | Gradient Boosting                                         |

---

## ✅ 2. Exemplos de Classificação e Regressão com GBM (`GradientBoostingClassifier` e `Regressor`)


In [1]:
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.datasets import load_iris, make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

# CLASSIFICAÇÃO
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("✅ Acurácia (Classificação):", accuracy_score(y_test, y_pred))

# REGRESSÃO
Xr, yr = make_regression(n_samples=200, n_features=4, noise=0.1, random_state=42)
Xr_train, Xr_test, yr_train, yr_test = train_test_split(Xr, yr, random_state=42)

reg = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
reg.fit(Xr_train, yr_train)
yr_pred = reg.predict(Xr_test)
print("✅ Erro Quadrático Médio (Regressão):", mean_squared_error(yr_test, yr_pred))

✅ Acurácia (Classificação): 1.0
✅ Erro Quadrático Médio (Regressão): 951.4441426212903


## ✅ 3. Cinco hiperparâmetros importantes no Gradient Boosting (GBM)

| Hiperparâmetro     | Descrição                                                                                   |
|--------------------|----------------------------------------------------------------------------------------------|
| `n_estimators`     | Número total de árvores (modelos fracos) treinadas sequencialmente.                         |
| `learning_rate`    | Controla o impacto de cada árvore na predição final. Valores menores exigem mais árvores.  |
| `max_depth`        | Profundidade máxima de cada árvore. Limita a complexidade e ajuda a evitar overfitting.     |
| `subsample`        | Proporção dos dados de treino usados em cada árvore (Stochastic GBM se < 1.0).              |
| `loss`             | Função de perda a ser minimizada (ex: `log_loss` para classificação, `squared_error` para regressão). |

---

### 🧠 Observações:

- **`n_estimators`** e **`learning_rate`** têm uma relação inversa: quanto menor o learning rate, mais árvores geralmente são necessárias.
- **`max_depth`** controla a complexidade de cada árvore — profundidades maiores capturam padrões mais complexos, mas podem causar overfitting.
- **`subsample < 1.0`** ativa o modo *Stochastic GBM*, introduzindo aleatoriedade e melhorando a capacidade de generalização.
- **`loss`** depende da tarefa: `log_loss` é padrão para classificação, `squared_error` para regressão.

---

### 💡 Dica prática:

Comece com:

```python
GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    subsample=0.8,
    loss='log_loss',
    random_state=42
)


## ✅ 4. Ajuste automático com GridSearchCV

In [2]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [2, 3, 5],
    'subsample': [0.6, 0.8, 1.0]
}

grid = GridSearchCV(
    GradientBoostingClassifier(random_state=42),
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)

grid.fit(X_train, y_train)
print("🎯 Melhores parâmetros:", grid.best_params_)
print("✅ Acurácia com modelo ajustado:", accuracy_score(y_test, grid.best_estimator_.predict(X_test)))


Fitting 5 folds for each of 54 candidates, totalling 270 fits
🎯 Melhores parâmetros: {'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 50, 'subsample': 0.6}
✅ Acurácia com modelo ajustado: 1.0


## ✅ 5. Diferença entre GBM tradicional e Stochastic GBM

📖 Segundo o artigo **"Stochastic Gradient Boosting"** de Jerome Friedman (1999):

### 🎯 A maior diferença entre os dois algoritmos está no uso de **subamostragem aleatória** dos dados.

| Característica             | GBM Tradicional                              | Stochastic GBM                                  |
|----------------------------|-----------------------------------------------|-------------------------------------------------|
| Dados usados por iteração  | 100% dos dados de treino                      | Apenas uma fração dos dados (ex: 50% ou 80%)    |
| Parâmetro chave            | —                                             | `subsample < 1.0` no `GradientBoosting`         |
| Variação/aleatoriedade     | Nenhuma (determinístico)                      | Introduzida via amostragem sem reposição        |
| Vantagens                  | Aprendizado mais estável                     | Reduz overfitting e melhora generalização       |
| Nome dado por Friedman     | —                                             | **Stochastic Gradient Boosting**                |

---

### ✅ Em resumo:

> **A principal diferença entre o GBM tradicional e o Stochastic GBM é que o Stochastic utiliza apenas uma amostra aleatória dos dados de treino em cada iteração (definida pelo parâmetro `subsample`), enquanto o GBM tradicional usa todos os dados.**

Essa estratégia de aleatoriedade controlada **ajuda a reduzir o overfitting** e torna o modelo mais **robusto em dados ruidosos ou com muitos exemplos similares**.

---